In [1]:
import pandas as kungfupanda
from os import listdir
from os.path import isfile, join
import os
import re

# Notebook for Updating Stats and Links on README
This is just a helper file I use to automatically link my solution files to the [README.md](README.md) page, formatting the links and details into markdown tables and calculating "solved" stats in the process. Just makes my life easier and I find it satisfying seeing the links lol. 

Just something I whipped up for fun. :)

In [2]:
leetcodePath = 'my-submissions/'
leetcodeFiles = [x for x in listdir(leetcodePath) if isfile(join(leetcodePath, x))]

contestFolders = [x for x in listdir(leetcodePath) if not isfile(join(leetcodePath, x))]
contestLeetcodeFiles = []
for contestFolder in contestFolders :
    contestLeetcodeFiles.extend([(contestFolder, fileName) for fileName in listdir(join(leetcodePath, contestFolder)) 
                                                            if isfile(join(leetcodePath, contestFolder, fileName))])

leetcodeFiles.sort()
contestLeetcodeFiles.sort()

print(leetcodeFiles)
print(f'Total of {len(leetcodeFiles)} files found.')

print(contestLeetcodeFiles)
print(f'Total of {len(contestLeetcodeFiles)} contest files found.')

['e1 - brute force.java', 'e1.java', 'e1.py', 'e1002 Daily.py', 'e1030.py', 'e1046.py', 'e1051.c', 'e1051.py', 'e1068.sql', 'e111.c', 'e118.py', 'e1184.py', 'e125.c', 'e125.py', 'e1252 v2 bools.c', 'e1252.c', 'e1317.c', 'e1351 v1 inefficient.java', 'e1351 v2 optimized.java', 'e1351.py', 'e14.java', 'e141 sets.py', 'e141 two pointer race.c', 'e1426.c', 'e1436.java', 'e1455 v2 less efficient.py', 'e1455.py', 'e1460.c', 'e1470 v2.c', 'e1470.c', 'e1474.c', 'e1474.java', 'e1502.c', 'e1502.py', 'e1684.c', 'e1684.py', 'e1700 v1.py', 'e1700 v2.py', 'e1736.c', 'e1748.c', 'e175.py', 'e175.sql', 'e181.sql', 'e182.sql', 'e1822.c', 'e183.sql', 'e1929.c', 'e1929.java', 'e1929.py', 'e196.py', 'e196.sql', 'e197.sql', 'e20.java', 'e2011.c', 'e2037 v1 preserving data.c', 'e2037 v2 nonpreserving.c', 'e21.java', 'e2206 v2 without double mallocing.c', 'e2206.c', 'e2206.py', 'e2248.py', 'e2389.py', 'e2395.py', 'e2570 v1.c', 'e2570 v2 less mallocing.c', 'e26.java', 'e2678.java', 'e27.java', 'e27.py', 'e28.py

## Helper Methods

AddCase $\rightarrow$ takes information for a new question file and formats it accordingly for a row.

UpdateLanguage $\rightarrow$ if a question already has a solution, this is called instead to insert the new file link to the existing row details.

In [3]:
# Categories

categories = set(['Daily', 'Weekly Premium', 'Contest', 'Favourite'])

In [4]:
def addCase(level: str, 
            number: int, 
            title: str, 
            category: str,
            language: str,
            path: str) :

    # Level, Number, Title, Python, Java, MySQL, Other
    output = [level, number, title, category, '', '', '', '', '']
    path = f'[{language}](<{path}>)'
    
    match language.lower() :
        case 'python' | 'py':
            output[4] = path
        case 'java':
            output[5] = path
        case 'mysql' | 'sql' :
            output[6] = path
        case 'c' :
            output[7] = path
        case _:
            output[8] = path

    # return df
    return output

In [5]:
def updateLanguage(orig, language, path) :  
    index = -1  
    match language.lower() :
        case 'python' | 'py':
            index = 4
        case 'java':
            index = 5
        case 'mysql' | 'sql' :
            index = 6
        case 'c' :
            index = 7
        case _:
            index = 8

    path = f'[{language}](<{path}>)'
    orig[index] = f'{orig[index]}, {path}' if not orig[index] == '' \
                                           else path

    return orig

In [6]:
def updateCategory(orig, category) : 
    if category.lower() in orig[3].lower() :
        return orig

    orig[3] = f'{orig[3]}, {category}' if not orig[3] == '' \
                                      else category
    return orig

# Parsing Files
Question file parsing occurs here. It organizes it into 3 different lists, separated by difficulty and sorted by question number afterwards.

In [7]:
easyQuestions = [] 
mediumQuestions = []
hardQuestions = []

questionsVisited = set()

counter = {}
moreThanOnce = 0

In [8]:
def parseCase(leetcodeFile: str, altTitle: str, subFolderPath: str, contest: bool) -> bool:
    level = leetcodeFile[0].lower()
    number = int(re.sub("[^0-9]", "", leetcodeFile.split(' ')[0])) # Strips all non-numeric characters and any that 
    title = ''                                                     # follow the question number e.g. e123 v1 becomes 123
    category = ''
    language = leetcodeFile[leetcodeFile.find('.') + 1:]
    path = join(leetcodePath, subFolderPath, leetcodeFile).replace("\\", "/")
    # print(path)

    if len(altTitle) > 0 :
        title = altTitle
    if contest == True :
        temp = re.findall('q\d{1}', leetcodeFile)
        if not len(temp) == 0 :
            title += ' - ' + temp[0]
        print(title)

    if contest :
        category = 'Contest'
    else :
        for cat in categories :
            if cat.lower() in leetcodeFile.lower() :
                category = cat
                break


    counter[(level, language)] = counter.get((level, language), 0) + 1 # For later reference

    if number in questionsVisited : # if solution already found for this question
        global moreThanOnce
        moreThanOnce += 1

        match level :
            case 'e' :
                for i in range(len(easyQuestions)) :
                    if easyQuestions[i][1] == number :
                        easyQuestions[i] = updateLanguage(easyQuestions[i], language, path)
                        break
            case 'm' :
                for i in range(len(mediumQuestions)) :
                    if mediumQuestions[i][1] == number :
                        mediumQuestions[i] = updateLanguage(mediumQuestions[i], language, path)
                        break
            case 'h' :
                for i in range(len(hardQuestions)) :
                    if hardQuestions[i][1] == number :
                        hardQuestions[i] = updateLanguage(hardQuestions[i], language, path)
                        break
        
        if category != '' :
            match level :
                case 'e' :
                    for i in range(len(easyQuestions)) :
                        if easyQuestions[i][1] == number :
                            easyQuestions[i] = updateCategory(easyQuestions[i], category)
                            break
                case 'm' :
                    for i in range(len(mediumQuestions)) :
                        if mediumQuestions[i][1] == number :
                            mediumQuestions[i] = updateCategory(mediumQuestions[i], category)
                            break
                case 'h' :
                    for i in range(len(hardQuestions)) :
                        if hardQuestions[i][1] == number :
                            hardQuestions[i] = updateCategory(hardQuestions[i], category)
                            break

        return True

    questionsVisited.add(number)

    match level :
        case 'e' :
            easyQuestions.append(addCase('Easy', number, title, category, language, path))
        case 'm' :
            mediumQuestions.append(addCase('Medium', number, title, category, language, path))
        case 'h' :
            hardQuestions.append(addCase('Hard', number, title, category, language, path))

    return True

In [9]:
# Parsing primary files
for leetcodeFile in leetcodeFiles :
    parseCase(leetcodeFile, '', '', False)

In [10]:
# Parsing contest files & folforders

for leetcodeContestFile in contestLeetcodeFiles :
    contestFolder, leetcodeFile = leetcodeContestFile
    parseCase(leetcodeFile, contestFolder, contestFolder, True)

bw contest 132 - q1
bw contest 132 - q2
w contest 400 - q1
w contest 400 - q2
w contest 401 - q1
w contest 401 - q2
w contest 401 - q3
w contest 401 - q3
w contest 401 - q3


In [11]:
print(easyQuestions)
print(mediumQuestions)
print(hardQuestions)

[['Easy', 1, '', '', '[py](<my-submissions/e1.py>)', '[java](<my-submissions/e1 - brute force.java>), [java](<my-submissions/e1.java>)', '', '', ''], ['Easy', 1002, '', 'Daily', '[py](<my-submissions/e1002 Daily.py>)', '', '', '', ''], ['Easy', 1030, '', '', '[py](<my-submissions/e1030.py>)', '', '', '', ''], ['Easy', 1046, '', '', '[py](<my-submissions/e1046.py>)', '', '', '', ''], ['Easy', 1051, '', '', '[py](<my-submissions/e1051.py>)', '', '', '[c](<my-submissions/e1051.c>)', ''], ['Easy', 1068, '', '', '', '', '[sql](<my-submissions/e1068.sql>)', '', ''], ['Easy', 111, '', '', '', '', '', '[c](<my-submissions/e111.c>)', ''], ['Easy', 118, '', '', '[py](<my-submissions/e118.py>)', '', '', '', ''], ['Easy', 1184, '', '', '[py](<my-submissions/e1184.py>)', '', '', '', ''], ['Easy', 125, '', '', '[py](<my-submissions/e125.py>)', '', '', '[c](<my-submissions/e125.c>)', ''], ['Easy', 1252, '', '', '', '', '', '[c](<my-submissions/e1252 v2 bools.c>), [c](<my-submissions/e1252.c>)', ''], 

In [12]:
# Sorting by question number
easyQuestions = sorted(easyQuestions, key=lambda x: x[1])
mediumQuestions = sorted(mediumQuestions, key=lambda x: x[1])
hardQuestions = sorted(hardQuestions, key=lambda x: x[1])

# DataFrames
Conversion into DataFrames and declaration of respective column headers occurs here.

In [13]:
columns = ['Level', 
           'Number', 
           'Title', 
           'Category',
           'Python', 
           'Java', 
           'MySQL', 
           'C',
           'Other'
           ]
typeClarification = {'Level': 'str', 
                     'Number': 'int', 
                     'Title': 'str', 
                     'Category': 'str',
                     'Python': 'str', 
                     'Java': 'str', 
                     'MySQL': 'str', 
                     'C': 'str',
                     'Other': 'str'
                     }

dfEasy = kungfupanda.DataFrame(data=easyQuestions, columns=columns)
dfMedium = kungfupanda.DataFrame(data=mediumQuestions, columns=columns)
dfHard = kungfupanda.DataFrame(data=hardQuestions, columns=columns)

dfEasy = dfEasy.astype(typeClarification)
dfMedium = dfMedium.astype(typeClarification)
dfHard = dfHard.astype(typeClarification)


In [14]:
# Helper method for outputing just to make code cleaner
def subLevel(level: str) -> str :
    match level.lower() :
        case 'e' :
            return 'Easy'
        case 'm' :
            return 'Medium'
        case 'h' :
            return 'Hard'
        case _ :
            return 'Unknown'

In [15]:

print(counter, sum(counter.values()))
print(len(questionsVisited))
print(moreThanOnce)

rows = ['e', 'm', 'h']
columns = sorted(list(set([x[1] for x in counter.keys()])))

# Initialization of stats table + column header initialization
statsMatrix = [[None] * (len(columns) + 2 + 1)] * (len(rows) + 2)
statsMatrix[0] = [''] + [f'**{x.title()}**' for x in columns] + ['**Total**', '**Total Unique**']

{('e', 'java'): 27, ('e', 'py'): 52, ('e', 'c'): 30, ('e', 'sql'): 14, ('h', 'py'): 5, ('h', 'c'): 1, ('m', 'py'): 77, ('m', 'c'): 19, ('m', 'java'): 18, ('m', 'sql'): 4, ('m', 'cpp'): 1, ('m', 'js'): 1, ('m', 'kt'): 1, ('m', 'rb'): 1, ('m', 'rs'): 1} 252
145
107


## Category parsing
Going row by row to calculate the respective values :v

In [16]:
temp = [easyQuestions, mediumQuestions, hardQuestions] # For summing number of unique question later

for i in range(1, len(rows) + 1) :
    statsMatrix[i] = [f'**{subLevel(rows[i - 1])}**'] \
                     + [counter.get((rows[i - 1], columns[j]), 0) for j in range(len(columns))] \
                     + [f'*{sum([counter.get((rows[i - 1], x), 0) for x in columns])}*'] \
                     + [f'*{len(temp[i - 1])}*']


# Total sums at bottom for each language and TOTAL TOTAL
# Total including double counts for languages
''' Line by line:   Row Header
                    Sum for each language
                    Total everything
                    Total unique everything
'''
statsMatrix[-1] = ['**Total**'] \
                  + [f'*{sum([counter.get((x, y), 0) for x in rows])}*' for y in columns] \
                  + [f'**{sum(counter.values())}**'] \
                  + [f'**{sum([len(x) for x in temp])}**']
temp = None

In [17]:
# Conversion to DataFrame
dfColumns = statsMatrix[0]
statsMatrix = [statsMatrix[i] for i in range(1, len(statsMatrix))]

statsMatrixDf = kungfupanda.DataFrame(data=statsMatrix, columns=dfColumns)
print(statsMatrixDf.to_markdown(index=False))

|            | **C**   | **Cpp**   | **Java**   | **Js**   | **Kt**   | **Py**   | **Rb**   | **Rs**   | **Sql**   | **Total**   | **Total Unique**   |
|:-----------|:--------|:----------|:-----------|:---------|:---------|:---------|:---------|:---------|:----------|:------------|:-------------------|
| **Easy**   | 30      | 0         | 27         | 0        | 0        | 52       | 0        | 0        | 14        | *123*       | *79*               |
| **Medium** | 19      | 1         | 18         | 1        | 1        | 77       | 1        | 1        | 4         | *123*       | *62*               |
| **Hard**   | 1       | 0         | 0          | 0        | 0        | 5        | 0        | 0        | 0         | *6*         | *4*                |
| **Total**  | *50*    | *1*       | *45*       | *1*      | *1*      | *134*    | *1*      | *1*      | *18*      | **252**     | **145**            |


# Outputing to README File
Takes all the above and overwrites the current [README.md](README.md) file with the data calculated above.

Inputs values in order of:
- Profile link
- Stats
- Stat clarification
- Question link tables Easy-Medium-Hard

Uses the built-in DataFrame `.to_markdown()` for outputting.

In [18]:
readmePath = 'README.md'

with open(readmePath, 'w') as file :
    file.write('# LeetCode Records\n\n')

    file.write('Profile: [Zanger](https://leetcode.com/u/Zanger/)\n\n')

    file.write('> *Note: if there are multiple files, it\'s likely a case of me having multiple solutions.*\n\n')

    file.write('## Stats by Language and Level\n\n')
    file.write(statsMatrixDf.to_markdown(index=False))
    file.write(f'\n\nQuestions done in multiple languages:\t{moreThanOnce}\n<br>')
    file.write(f'\nUnique questions done:\t\t**{int(re.sub("[^0-9]", "", statsMatrix[len(statsMatrix) - 1][len(statsMatrix[0]) - 2])) - moreThanOnce}**')

    file.write('\n\n\n')


    file.write('## Easy\n')
    file.write(dfEasy.to_markdown(index=False))

    file.write('\n\n## Medium\n')
    file.write(dfMedium.to_markdown(index=False))

    file.write('\n\n## Hard\n')
    file.write(dfHard.to_markdown(index=False))